In [1]:
# Induction motor calculations and assumptions
import numpy as np


#Assumptions
Poles = 4
V_Dc = 230                  # input Dc voltage in V
m = 0.9                     # modulation index,usually less than 1 
OP_Pwr = 3000               # output power of motor - 3Kw 
Power_fac = 0.8             # power factor - cos(phi)
n_eff = 0.75                # typical efficiency of  1-5 hp motor
K_ws = 0.95                 # stator winding factor kw = kp * kd * ks
freq_hz = 120 
Dia = 0.07                  # Inner diameter of stator diameter in m L/tau_p = 1.5
Len = 0.08                  # length in m
ac = 15000                  # specific electric loading
tau_s1 = 0.0091             # slot pitch 15mm - 20mm
Del_1 = 4                   # current density in stator conductor 4A/mm2
del_bar = 5                 # cuurent density in rotor bar 5-7 A/mm2
mu_0 = (4*np.pi * 10**(-7)) #permeability of free space


def Calc_mag_flux(poles, pow_w, freq):
    mf = (0.00145 + (0.003 / poles)) * (np.sqrt((0.08 * pow_w) / freq))
    return mf 

def Calc_mag_flux_density(poles,pwr, freq, dia, length):
    b = (poles * Calc_mag_flux(poles,pwr,freq)) / (2 * dia * length) 
    return b
def Calc_air_gap(dia, length):
    lag = 0.2 + 2 * np.sqrt(dia*length)
    return lag

V_Ac_llrms = round((0.8165 / np.sqrt(2)) * V_Dc * m)
V_Ac_phiRms = round((V_Ac_llrms / np.sqrt(3)))          #Vph1

print("V_Ac_llrms = ", V_Ac_llrms, "V")
print("V_Ac_phiRms = Vph1 = ", V_Ac_phiRms,"V")


I_rms = ((OP_Pwr / 1000 )* 10**(3))/( 3 *  V_Ac_phiRms * Power_fac * n_eff )    #Iph1
print("I_rms =", I_rms,"A")

    
C_OpCoeff = 17.35 * 10**(-5) * Calc_mag_flux_density(Poles, OP_Pwr, freq_hz, Dia, Len) * ac * Power_fac * n_eff
print("C_OpCoeff = ", C_OpCoeff)

print("max mag flux = ", Calc_mag_flux(Poles, OP_Pwr, freq_hz))
print("mag flux den = ", Calc_mag_flux_density(Poles, OP_Pwr, freq_hz, Dia, Len), "wb/m2")

print("air gap = ", Calc_air_gap(Dia,Len))

S1 = round(((np.pi * Dia) / tau_s1))
if (S1 % 4 != 0):
    temp = round(S1 / 4)
    S1 = 4 * temp

print("No of stator slots = ", S1)

q1 = round(S1 / (3 * Poles),1)  #no. of slots per pole per phase
print("no. of slots per pole per phase = ", q1)

N_ph1 = round(V_Ac_phiRms / (4.44 * K_ws * freq_hz * Calc_mag_flux(Poles, OP_Pwr, freq_hz)))
print("No of turns per phase = ", N_ph1)

Z1 = round(6 * N_ph1)  #total no of conductors
print("total no of conductors = ", Z1)

N_c1 = round(Z1 / S1)
print("No of conductors per slot = ", N_c1)

F_c1 = I_rms / Del_1
print("sectional area of stator conductor = ", F_c1, "in mm2")

# To be implemented
    #Stator slot design
    #Length of mean turns
    #Resistance of stator winding per phase
    #Total copper loss in the stator winding
    #Flux density in stator tooth; MMF; #Maximum flux density in stator tooth should not exceed 1.8T
    #Depth (Height) of stator yoke 
    #Outer dia of IM 
    #Estimation of iron losses in stator


tau_p = (np.pi * Dia) / Poles
L_mt = 2 * Len + 2.3 * tau_p + 0.24
print("Length of mean turns = ", L_mt)

R_ph1 = 0.021 * 10**(-6) * (L_mt / (F_c1 * 10**(-6))) * N_ph1
print("Resistance of stator winding per phase = ", R_ph1, "ohm")

print("Total copper loss in stator winding = ", 3 * I_rms**2 * R_ph1, "W")


#Rotor Design
print("************************Rotor Design************************\n")

q2 = q1-0.17                                                 #assume; for 24 slot stator
S2 = round(3 * q2 * Poles)
print("No of rotor slots s2 = ", S2)

i2_bar = (0.85 * I_rms * 6 * N_ph1) / S2
print("rotor current = ", i2_bar)

Z_2bar = S2
print("No of rotor bars for cage motors equals No of rotor slots = ", S2)

I2_endring = (S2 * i2_bar) / (np.pi * Poles)
print("rotor endring current = ",I2_endring)

#Size of rotor conductors:
F_c2bar = i2_bar / del_bar
print("sectional area of rotor bar = ", F_c2bar)

F_c2endring = I2_endring / del_bar
print("sectional area of rotor endring = ",F_c2endring)


# Assumptions or values to be assigned
    # Flux density in rotor tooth : delta, h_t2, K_i, bs_2, l_tooth
    # Rotor copper loss in cage rotor: d_2bar
    # Slip calculation: Mech_pwr_op
    # Effective air gap length: b_01, b_02 ,b_s2, b_s1
    # Estimation of magnetizing current and no load current: A_Ty, A_T2ht1, A_T2del, A_T2ht2, A_Try, K_pd1, V_ph1, P_i
    # Estimation of ideal blocked rotor current: R1, R2, X1, X2
    # Estimation of leakage reactance: x, dx, h1, h2, h3,h4, h5
    # Rotor slot leakage reactance: h, K_pd2
    # Overhang leakage reacance: L_0 - Length of conductor in overhang, K_s - Slot leakage factor
    # Belt or Harmonic leakage reactance: K_h1 - factor of stator, K_h2 - factor of rotor


#Flux density in rotor tooth
D1_3 = Dia - (2 * delta) - (2/3 * h_t2) * 2
print("Dia of rotor at 1/3rd of tooth height = ", D1_3)

tau_s1_3 = (np.pi * D1_3) / S2
print("slot pitch of rotor at 1/3rd of tooth height = ", tau_s1_3)

b_1_3 = tau_s1_3 - bs_2 
print("width of tooth at 1/3rd of tooth height = ", b_1_3)

A_1 = b_1_3 * K_i * l_tooth
print("area of one stator tooth at 1/3rd of tooth height = ", A_1)

A_st = A_1 * (S2 / Poles)
print("area of all stator teeth under one pole = ", A_st)

phi_1 = A_st * b_1_3
print("mean flux density = ", phi_1)

#Rotor copper loss
Res_1_bar = 0.021 * 10**(-6) * (Len / F_c2bar)
print("Resistance in one rotor bar = ", Res_1_bar)

Cu_loss_bar = S2 * Res_1_bar * i2_bar**2
print("Copper loss in all rotor bars = ", Cu_loss_bar)

Res_endring = 0.021 * 10**(-6) * (ni.pi * (Dia - 2 * delta - 2 * d_2bar)) /  F_c2endring
print("Resistance of one end ring = ", Res_endring)

Cu_loss_endring = 2 * Res_endring * I2_endring**2
print("Copper loss in end rings = ", Cu_loss_endring)

T_Cu_loss = Cu_loss_endring + Cu_loss_bar
print("total Copper loss = ", T_Cu_loss)

#SLip = Rotor Cu loss / Mech Power Output + Losses
#Losses = Rotor Iron Loss (Negligible) + Rotor Cu loss + F & W loss
#Slip = 0.5% for small motors; 2.5% - 3.5% for medium motors; 1% - 1.5% for large motors 
slip = 2.5
print("slip = ", slip)

#Effective air gap length
tau_sg1 = (np.pi * Dia) / S1
K_01 = 1 / (1 + 5 *(delta / b_s1))
K_c1 = tau_sg1 / (tau_sg1 - (K_01 * b_01))
print("Gap Contraction factor for stator = ", K_c1)

tau_sg2 = (np.pi * (Dia - 2 * delta)) / S2
K_02 = 1 / (1 + 5 *(delta / b_s2))
K_c2 = tau_sg2 / (tau_sg2 - (K_02 * b_02))
print("Gap Contraction factor for rotor = ", K_c2)

del_ag = K_c1 * K_c2 * delta
print("Effective air gap length = ", del_ag)

#FLUX DENSITY DISTRIBUTION
B_30deg = 1.35 * Calc_mag_flux_density(Poles, OP_Pwr, freq_hz, Dia, Len)
print("FLUX DENSITY at 30 deg from direct axis = ", B_30deg)

#MMF REQUIRED IN AIR GAP 
A_Tdel = (1 / mu_0) * B_30deg * del_ag
print("MMF REQUIRED IN AIR GAP = ", B_30deg)

#ESTIMATION OF MAGNETIZING CURRENT & NO LOAD CURRENT 
A_T30 = A_Ty + A_T2ht1 + A_T2del + A_T2ht2 + A_Try
print("AT at one pole = ", A_T30 / 2)

Im = A_T30 * (Poles / 2 ) * (1 / (1.17 * K_pd1 * N_ph1))
Ic = P_i / (3 * V_ph1)
I0 = np.sqrt(Ic**2 + Im**2)
print("No load current = ", I0)

cos_phi0 = I0 / Ic
print("No load power factor = ", cos_phi0)

#ESTIMATIONS OF IDEAL BLOCKED ROTOR CURRENT
R2_dash = ((Kpd1 * N_ph1) / (Kpd2 * N_ph2))**2 * R2
R01 = R1 + R2_dash
print("Total resistance referred to stator = ", R01)

X2_dash = ((Kpd1 * N_ph1) / (Kpd2 * N_ph2))**2 * X2
X01 = X1 + X2_dash
print("Total leakage reactance referred to stator = ", X01)

Isc_ideal = V_ph1 / (np.sqrt(R01**2 + X01**2))
print("IDEAL BLOCKED ROTOR CURRENT = ", Isc_ideal)

cos_phisc = R01 / (np.sqrt(R01**2 + X01**2))
print("power factor stator current = ", cos_phisc)


#ESTIMATION OF LEAKAGE REACTANCE
#Stator slot leakage reactance
#For 1 layer winding

I_c1 = i2_bar                              #current through rotor bar
M_x = (N_c1 * I_c1 * dx) / h1
print("MMF at distance x = ", M_x)

permeance = mu_0 * ((Len * dx) / b_s1)
print("permeance of path = ", permeance)

D_phi_x = M_x * permeance
print("leakage flux through the elementary path of thickness dx & height x = ", D_phi_x)

D_psi_x = (N_c1 * x) / h1 * D_phi_x
print("Leakage flux linkages associated with elementary path = ", D_psi_x)

Psi_h1 = mu_0 * N_c1**2 * I_c1 * Len* ( h1 / (3 * b_s1))
print("flux linkages in height h1 = ", Psi_h1)

Psi_h2 = mu_0 * N_c1**2 * I_c1 * Len* ( h2 / b_s1)
print("flux linkages in height h2 = ", Psi_h2)

Psi_h3 = mu_0 * N_c1**2 * I_c1 * Len* ( (2 * h3) / (b_s1 + b_01))
print("flux linkages in height h3 = ", Psi_h3)

Psi_h4 = mu_0 * N_c1**2 * I_c1 * Len* ( h4 / b_01)
print("flux linkages in height h3 = ", Psi_h4)

lambda_s = mu_0 * ((h1 / (3 * b_s1)) + ( h2 / b_s1) + ( (2 * h3) / (b_s1 + b_01)) + ( h4 / b_01))
print("specific slot permeance = ", lambda_s)

Psi_s1 = N_c1**2 * I_c1 * Len * lambda_s
print("Total slot leakage flux = ", Psi_s1)

L_s1 =  N_c1**2 * Len *  lambda_s
print("Slot leakage inductance = ", L_s1)

print("Slot leakage inductance per phase = ", Poles * q1 * N_c1**2 * Len * lambda_s)

X_s1 = 8 * np.pi * freq_hz * (N_ph1**2 / (Poles * q1)) * Len * lambda_s
print("Slot leakage reactance per phase for 1 layer winding = ", X_s1)

#For 2-Layer winding
X_s2 = 8 * np.pi * freq_hz * (N_ph1**2 / (Poles * q1)) * Len * mu_0 * (((2 *h1) / (3 * b_s1)) + ( h2 / ( 4 * b_s1)) + ( h3 / b_s1 ) + ( (2* h4) / (b_s1 + b_01)) + (h5 / b_01))
print("Slot leakage reactance per phase  for 2 layer winding = ", X_s2)

X1 = X_s2                                                   #assign X1 = X_s2 or X1 = X_s1;choose X1 based on 1 layer or two layer winding

#Rotor slot leakage reactance

w0 = 0.2                                                    #assume w0 = b02 = (0.2 - 0.4)d_2bar
lambda_r = mu_0 * (0.623 + (h / w0))
print("Rotor Specific Permeance = ", lambda_r)

N_ph2 = S2 / 6

X2 = 8 * np.pi * freq_hz * (N_ph2**2 / (Poles * q2)) * Len * lambda_r
print("Rotor reactance per phase = ", X2)

X2_dash = ((K_pd1 * N_ph1)/(K_pd2 * N_ph2))**2 * X2
print("Rotor resistance referred to stator = ", X2_dash)

#Overhang leakage reactance
lambda_0 = mu_0 * K_s * (tau_p**2 / (L_0 * np.pi * tau_s1))
print("Overhang Specific Permeance = ", lambda_0)

X_0 = 8 * np.pi * freq_hz * (N_ph1**2 / (Poles * q1)) * L_0 * lambda_0
print("Overhang leakage reactance = ", X_0)

#Zigzag leakage reactance 
X_m = V_ph1 / Im
SP_1 = S1 / Poles
SP_2 = S2 / Poles

X_z = (5/6) * X_m * ((1 / SP_1**2) + (1/SP_2**2))
print("Zigzag leakage reactance = ", X_z)

#Differential or Belt or Harmonic leakage reactance 
X_h = X_m * (K_h1 * K_h2)
print("Belt or Harmonic leakage reactance = ", X_h)

print("Total leakage reactance referred to the stator side = ", X_s1 + X2_dash + X_0 + X_z + X_h)



V_Ac_llrms =  120.0 V
V_Ac_phiRms = Vph1 =  69.0 V
I_rms = 24.154589372 A
C_OpCoeff =  1.73508851815
max mag flux =  0.00311126983722
mag flux den =  1.11116779901 wb/m2
air gap =  0.349666295471
No of stator slots =  24
no. of slots per pole per phase =  2.0
No of turns per phase =  44.0
total no of conductors =  264.0
No of conductors per slot =  11.0
sectional area of stator conductor =  6.038647343 in mm2
Length of mean turns =  0.5264491043069892
Resistance of stator winding per phase =  0.0805542938261 ohm
Total copper loss in stator winding =  140.996803588 W
************************Rotor Design************************

No of rotor slots s2 =  22
rotor current =  246.376811594
No of rotor bars for cage motors equals No of rotor slots =  22
rotor endring current =  431.332961713
sectional area of rotor bar =  49.2753623188
sectional area of rotor endring =  86.2665923426


NameError: name 'delta' is not defined